In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# All skl imports go here
from sklearn import tree   # Decision Trees
from sklearn import svm    # svm
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import sklearn as skl

from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam, Optimizer
from torch.nn import CrossEntropyLoss
import torch

from neuralnet import NeuralNetwork

# Data Loading

In [ ]:
NUM_CLASSES = 6
CLASSES = ["sadnesss", "joy", "love", "anger", "fear"]

In [ ]:
# Load all data
train_data = pd.read_csv("data/training_labse.csv")
test_data = pd.read_csv("data/test_labse.csv")
validation_data = pd.read_csv("data/validation_labse.csv")

# Separate X's and y's from each other
FEATURE_COLUMNS = [x for x in train_data if x.startswith("_e")]
LABEL_COLUMN = "label"

X_train = train_data[FEATURE_COLUMNS]
Y_train = train_data[LABEL_COLUMN]

X_test = test_data[FEATURE_COLUMNS]
Y_test = test_data[LABEL_COLUMN]

X_val = validation_data[FEATURE_COLUMNS]
Y_val = validation_data[LABEL_COLUMN]

# These are used to run cross validation
X_train_val = pd.concat([X_train, X_val]) 
Y_train_val = pd.concat([Y_train, Y_test])

In [ ]:
# Perform pre-processing PCA on the training set
TARGET_EXPLAINED_VARIANCE = 0.95 

def perform_pca(dataset):
    pca = PCA(n_components= TARGET_EXPLAINED_VARIANCE)

    # Need to standardize the data frirst
    standardized = (dataset - dataset.mean(axis=0)) / dataset.std(axis = 0)

    pca.fit(X=standardized)
    dataset_reduced = pca.fit_transform(X=standardized)

    return pca, dataset_reduced



In [ ]:
pca_train, X_train_reduced = perform_pca(X_train)
pca_tran_val, X_train_val_reduced = perform_pca(X_train_val)

# K-Folds Cross Validation

In [ ]:

def k_folds_x_val(model): 
    cumulative_accuracy = 0
    cumulative_kappa = 0

    k_folds = KFold(n_splits=10)

    for i, (train_index, test_index) in enumerate(k_folds.split(X_train_val_reduced, Y_train_val)):
        model.fit(X_train_val_reduced[train_index], Y_train_val.iloc[train_index])
        
        Y_pred = model.predict(X_train_val_reduced[test_index])
        Y_true = Y_train_val.iloc[test_index]
        # Add all metrics here

        cumulative_accuracy += metrics.accuracy_score(Y_true, Y_pred)
        cumulative_kappa += metrics.cohen_kappa_score(Y_true, Y_pred)

    folds = k_folds.get_n_splits()
    print(f"Performed {folds}-fold cross validation")
    print(f"Average accuracy {cumulative_accuracy / folds}")
    print(f"Average Kappa {cumulative_kappa / folds}")


# Grid Search Hyperparameter Optimization

In [ ]:
def gridSearchHPO(model, search_space):
    grid_search = GridSearchCV(estimator=model,
                                param_grid=search_space,
                                scoring='accuracy',
                                cv=5,
                                verbose=3,
                                error_score='raise',
                                n_jobs=-1,  # -1 means max amount
                                )
    return grid_search

# Decision Trees

In [ ]:
decision_tree_model = tree.DecisionTreeClassifier(
    criterion ='entropy', 
    splitter = 'best',
    max_depth = 256, 
    max_features = 'sqrt',
    max_leaf_nodes = 80,
)

In [ ]:
# ['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 
# 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter']
dt_search_space = {
    # 'ccp_alpha':[0.1, 0.2, 0.4, 0.5],
    'max_leaf_nodes':[71, 72, 73, 74, 75, 76, 77, 78, 79, 80], 
    # 'min_impurity_decrease':[1.0, 0.5, 1.5, 2.0], # float
    # 'min_weight_fraction_leaf':[0.1, 0.2, 0.4, 0.5],
    'max_depth':[255, 256, 257, 258, 259, 260], 
    # 'max_features':["log2", "sqrt"], 
    # 'min_samples_leaf':[0.1, 0.2, 0.4, 0.5], 
    # 'min_samples_split':[0.1, 0.2, 0.4, 0.5]
    }

model_dt = decision_tree_model.fit(X_train, Y_train)

gridsearch_dt = gridSearchHPO(model=model_dt, search_space=dt_search_space)

In [ ]:
gridsearch_dt.fit(X_train, Y_train)

In [ ]:
print("Best Score: {}".format(gridsearch_dt.best_score_))
print("Best params: {}".format(gridsearch_dt.best_params_))

# SVMs

In [ ]:
svm_classifier = svm.SVC(max_iter=-1)

#Train the model using the training sets
svm_classifier.fit(X_train, Y_train)

#Predict the response for test dataset
# k_folds_x_val(svm_classifier)

In [ ]:
svm_search_space={
    'C': [0.1, 1, 10, 100, 1000],
    'kernel': ['poly'], # poly - 'rbf', 'linear', 'sigmoid' 
    'degree': [3],  # 3
    'gamma': ['scale'], #'auto',  1, 0.1, 0.01, 0.001, 0.0001, 
    'coef0': [0.0, 0.1, 0.2, 0.5, 0.8, 1.0], 
    'shrinking': [True], 
    'probability': [False], 
    'tol': [0.1],  
    'class_weight': [None], 
    'decision_function_shape': ['ovr'], # 'ovo'
}

gridsearch_svm = gridSearchHPO(svm_classifier, svm_search_space)

In [ ]:
gridsearch_svm.fit(X_train, Y_train)

In [ ]:
print("Best Score: {}".format(gridsearch_svm.best_score_))
print("Best params: {}".format(gridsearch_svm.best_params_))

# Pytorch Setup

In [ ]:
# Pytorch specific constants
from torch.utils.data import TensorDataset, DataLoader
from utils.neuralnet import NeuralNetwork
from utils.trainer import training_loop, evaluate
import torch

BATCH_SIZE = 32
LEARNING_RATE = 1e-3

In [ ]:
# For pytorch specifically we should load data to the provided dataloader and dataset classes. 
# This handles the batching for us.

pt_train_set = TensorDataset(torch.Tensor(X_train_reduced), torch.Tensor(Y_train.to_numpy()).long())
pt_val_set = TensorDataset(torch.Tensor(pca_train.transform(X_val)), torch.Tensor(Y_val.to_numpy()).long())
pt_test_set = TensorDataset(torch.Tensor(pca_train.transform(X_test)), torch.Tensor(Y_test.to_numpy()).long())
pt_val_test_set = TensorDataset(torch.Tensor(pca_train.transform(X_val_test)), torch.Tensor(Y_val_test.to_numpy()).long())

pt_train_loader = DataLoader(
                    dataset=pt_train_set, 
                    batch_size=BATCH_SIZE, 
                    shuffle=True,
                )
pt_val_loader = DataLoader(
                    dataset=pt_val_set, 
                    batch_size=1, 
                    shuffle=True,
                )
pt_test_loader = DataLoader(
                    dataset=pt_test_set, 
                    batch_size=BATCH_SIZE, 
                    shuffle=True,
                )

pt_val_test_loader = DataLoader(
                    dataset=pt_val_test_set, 
                    batch_size=BATCH_SIZE, 
                    shuffle=True,
                )

# MLP

In [ ]:
# Initialize neural network
mlp = NeuralNetwork(X_train_reduced.shape[1], [], NUM_CLASSES)

training_loop(mlp, pt_train_loader, pt_val_loader, 10, LEARNING_RATE)

In [ ]:
evaluate(model=mlp, val_dl=pt_test_loader)

# RNN

# Encoder-Only Transformer

In [ ]:
#TODO: Add stuff for running the test set on the model